In [138]:
import matplotlib.pyplot as plt
import numpy as np
import skimage
from skimage import feature
import scipy
from scipy.stats import chisquare
from scipy.optimize import curve_fit
from scipy import stats
from scipy.optimize import least_squares
import cv2
import tifffile as tiff
import os
from scipy import signal
import time
from scipy.optimize import minimize
from scipy.stats import norm
from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from matplotlib import cm
import argparse

folder = 'Calib_10days/'
filename = 'Calibration_10days_'
filepath = folder+filename
filetype = '.tiff'

#filename = 'Distance_Simulation/'
params = {'legend.fontsize': '22',
          'figure.figsize': (10, 8),
         'axes.labelsize': '22',
         'axes.titlesize':'22',
         'xtick.labelsize':'22',
         'ytick.labelsize':'22'}


def getImageDescription(im): return ast.literal_eval(im.tag[270][0])
def getClockScalingFactor(): return (28/1000)/(1750000) 
def get_tiff_integration_time(im): return float(getImageDescription(im)["integration time"])*getClockScalingFactor()


#Going to use this to get rid of dodgy files with 0 integration time
#Courtesy of sam

def get_file_nums(folder): #function returns ordered list of numbers of files in folder
    _, dirs, files = next(os.walk(folder))
    nums = ([])
    for file in files:
        try: 
            nums.append(int(file.replace(filename, '').replace(filetype, '')))
        except: continue
    return sorted(nums)


def snip(image,chop=2): return np.array([l[chop:-chop-1] for l in image[chop:-chop-1]])
    #This chops L R U D chop number of elements (so Hough works)

def image_ret(file_num, col=0):
    full_img = tiff.imread(filepath+str(file_num)+'.tiff')[:,:,col]
    #am inverting and changing mm.tiff 
    #full_img = np.invert(full_img)
    return snip(full_img)


def line_eqn(line): 
    x0,x1,y0,y1 = line
    m = (y1-y0)/(x1-x0)
    c = y0
    return [m,c]

def find_intercept(line1, line2):
    m1,c1 = line1
    m2,c2 = line2
    x = (c1-c2) / (m2-m1)
    y = m1 * x + c1
    return [x,y]  

def cross_points(image,lines):
    #takes lines and works out where they cross 
    #returns list of [x1,y1], [x2,y2]... of where points are
    done_list =([])
    pts = ([])
    for i in range(len(lines)):
        for j in range(len(lines)):
            if i!=j and not any(point in done_list for point in [[i, j], [j, i]]):
                done_list.append([i,j])
                p1,p2 = find_intercept(lines[i],lines[j])
                if 0<p1<image.shape[0] and 0<p2<image.shape[1]:
                    pts.append([int(p1),int(p2)])
    return pts

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    sqr = np.zeros((4, 2), dtype = "float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    sqr[0] = pts[np.argmin(s)]
    sqr[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    sqr[1] = pts[np.argmin(diff)]
    sqr[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return sqr


def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    #specifying points
    # top-left, top-right, bottom-right, and bottom-left
    sqr = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
        # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, sqr)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

def sum_pixel(images): return images.sum(axis=0)
def avg_pv(numbers): return np.array([image_ret(i) for i in bckr_count]).mean(axis=0)

dark_img_folder = 'Define this folder'
bckgr_count = get_file_nums(dark_img_folder)
#assumes the filename is the same as for normal files
dark = avg_pv(bckgr_count)


print(get_file_nums(folder))



[1, 2, 3, 4, 5, 6, 8, 10]


In [ ]:
#import files
    #define image locations
    #define dark locations
#got to define threshold for mean of image when it is below a certain value
    #use average of pixel value for whole image
#use greater than threshold - not background
#minus dark from images to get irradiation
#irrad_images - dark

#didn't move LASSENA so should be same pts for all images